<a href="https://colab.research.google.com/github/MMathisLab/LINdoscope2023/blob/main/notebooks/DeepLabCut_LindoScope_Colab_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://camo.githubusercontent.com/71523c879284afa1c51b8489491f06d24d1ff02d75dca9f71078535ee6b1978e/68747470733a2f2f696d616765732e73717561726573706163652d63646e2e636f6d2f636f6e74656e742f76312f3537663664353163396637343536366635356563663237312f313632383235303030343232392d4b565944374a4a5648594546444a33324c39564a2f444c436c6f676f323032312e6a70673f666f726d61743d3130303077)

👋 This notebook is a modified copy from [Github](https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb), originally written by Mackenzie Mathis and contributors.

📝 This notebook accompanies Nath, Mathis et al. 2019 Nature Protocols https://www.nature.com/articles/s41596-019-0176-0, which is the DeepLabCut 2.0 version of  publication: Mathis et al 2018, https://doi.org/10.1038/s41593-018-0209-y. Original source: https://github.com/DeepLabCut/DeepLabCut

⚠️ It has been edited for the 2023 EMBO practical course LINdoscope: https://www.lindoscope.com.


This notebook illustrates how to use DeepLabCut and Colab to:
- load a pretrained model from the Lindoscope Setting
- analyze videos, filter data, create video w/keypoints


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut[tf]
!pip install tensorflow==2.10 # just to be sure

**(Be sure to click "RESTART RUNTIME" if it is displayed above above before moving on !)**

## Link the Linodscope Google Drive:

Note: we are going to work with the Mouse Body model, found here: https://drive.google.com/drive/u/0/folders/1ufvR2GP-TEVaR3Z4qnJgr2I1aCioAfND



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Lets be sure you have access to the drive! You may need to mount this to add a link in your "My Drive" directory.
# you should see the DLC_models directory:
%cd /content/drive/MyDrive/LINdoscope2023_analysis
%ls

/content/drive/.shortcut-targets-by-id/1MCbA_XWnvaUo_aH2Nu4A3Sg_UwDbwAia/LINdoscope2023_analysis
DLC_models/  docs/  README.txt  sample_data/  student_data/


## Import Dependencies

In [ ]:
import matplotlib
import os
import deeplabcut

Loading DLC 2.3.5...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Setup your project variables

Here, we will use Oliver's model. But, if you want to change this if you have a new model, please do!

- Modify variables **PretrainedModelFolderName** and **videofile_path** to names of a folder with a pretrained model and to the videos you would like to analyse, respectively.


In [ ]:
# What is the name of your DLC project with the trained model:
PretrainedModelFolderName = 'MouseBody-Barnstedt-2019-09-09'

# What is the video you want to first test?
videofile_path = '/content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/Basler acA780-75gm (22611479)_20190218_151620616.mp4'
length = 30 # desired length in seconds

# Edit these if needed
framerate = 75
outsuffix = '_short'
VideoType = 'mp4'


In [ ]:
# the Project/model config file:

path_config_file = '/content/drive/My Drive/LINdoscope2023_analysis/DLC_models/'+PretrainedModelFolderName+'/config.yaml'
path_config_file

## 🚨 Optional: shorten video for speed

- Here, we preload a video of a mouse on the 2P system.


In [ ]:
# First, a 💡 ProTip:for every function you can use "?" to learn more!
deeplabcut.ShortenVideo?

In [ ]:
deeplabcut.ShortenVideo(videofile_path, start = "00:00:00", stop = "00:00:10", outsuffix = "_short", outpath='/content/')


In [ ]:
video_path_file_short = '/content/drive/My Drive/LINdoscope2023_analysis/sample_data/2P_dCA1_treadmill/Basler acA780-75gm (22611479)_20190218_151620616_short.mp4'
video_path_file_short


## Start Analyzing videos with your pretrained model!
This function analyzes your videos.

The results are stored in hd5 file in the same directory where the video resides. The data is optionally also exported in comma-separated values format (.csv).

In [ ]:
deeplabcut.analyze_videos?

In [ ]:
# some variables you might consider setting across the next set of functions:

batchsize = 128 #as large as your GPU can handle
shuffle = 1 #typically default is 1.
video_to_use = videofile_path # or consider shorter video
save_as_csv = False #which is the default; I really recommend becoming friends with H5 files 🤗

In [ ]:
deeplabcut.analyze_videos(path_config_file, video_to_use, videotype=VideoType, shuffle=shuffle, save_as_csv=save_as_csv, batchsize=batchsize)

You can also now filter your data to smooth any small jitters:

In [ ]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype=VideoType, shuffle=1)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=1, filtered=True)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType, shuffle=1, filtered=True)